In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/MyDrive/ift6289-project/
!ls

Mounted at /content/drive
/content/drive/MyDrive/ift6289-project
'Copy of ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'Copy of ift6289-project-mt5-base-ENtoBN.ipynb'
 gemma_7b_finetune_template.ipynb
 hasan-etal-2020-low
 hasan-etal-2020-low.tar.bz2
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-ENtoBN.ipynb'
'ift6289-project-facebook nllb-200-distilled-600M.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-bn-en.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-en-mul.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-mul-en.ipynb'
'ift6289-project-meta-llama Llama-2-7b-hf.ipynb'
 ift6289-project-mt5-base-BNtoEN.ipynb
 ift6289-project-mt5-base-ENtoBN.ipynb
 ift6289-project-umt5-base-BNtoEN.ipynb
 ift6289-project-umt5-base-ENtoBN.ipynb
 logs
 results


In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install transformers
!pip install ml_things
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.8 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=ffe68ed64bcfed255be38c

In [3]:
from transformers import set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline, MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import Dataset, load_dataset, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm  # tqdm is great for progress bars
import sacrebleu
import numpy as np

# Load dataset from HuggingFace

In [4]:
set_seed(42)

dataset = load_dataset("csebuetnlp/BanglaNMT")

training_subset_size = 100000
dataset["train"] = dataset["train"].select(range(training_subset_size))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2379749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

# Specify the model checkpoint

In [5]:
model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    inputs = [ex for ex in examples['bn']]
    targets = [ex for ex in examples['en']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "en_XX"
tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 597
    })
})


# Look at some dataset samples

In [7]:
for i in range(3):  # Adjust the range as needed
    print(f"\nExample {i+1}:")
    print("Input (English):", tokenizer.decode(tokenized_datasets['train'][i]['input_ids'], skip_special_tokens=True))
    print("Label (Bengali):", tokenizer.decode(tokenized_datasets['train'][i]['labels'], skip_special_tokens=True))


Example 1:
Input (English): আমাকে সব কিছু জানাও
Label (Bengali): Just keep me informed.

Example 2:
Input (English): "এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...
Label (Bengali): It's bad, "Amor and Psyche" is, master.

Example 3:
Input (English): কাছানো
Label (Bengali): Overtake


# Load model and data collator

In [8]:
model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

# Load SacreBLEU metrics

In [9]:
bleu_metric = load_metric('bleu')
sacrebleu_metric = load_metric('sacrebleu')

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_labels = []
    for label in labels:
        label = label[label != tokenizer.pad_token_id]  # Remove padding token IDs
        decoded_label = tokenizer.decode(label, skip_special_tokens=True)
        decoded_labels.append(decoded_label)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # # Compute BLEU score
    # bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[decoded_labels])
    # Compute SacreBLEU score
    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare result dictionary
    results = {
        # "bleu": bleu_result['score'],
        "sacrebleu": sacrebleu_result['score']
    }

    return results

<ipython-input-9-d8f36cbc978d>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


# Define training arguments and fine-tune

In [10]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    # report_to="none",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# # Pre-training test results
baseline_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Baseline Test Metrics:", baseline_test_results.metrics)
baseline_validation_results = trainer.evaluate()
print("Baseline Validation Metrics:", baseline_validation_results)

# Fine-tune the model
print("\nFine-tuning the model...")
trainer.train()
print("Model fine-tuning complete.")

Baseline Test Metrics: {'test_loss': 3.220021963119507, 'test_sacrebleu': 7.1843089382168746, 'test_runtime': 124.0087, 'test_samples_per_second': 8.064, 'test_steps_per_second': 0.508}


Baseline Validation Metrics: {'eval_loss': 3.2606852054595947, 'eval_sacrebleu': 6.688215908407851, 'eval_runtime': 69.5402, 'eval_samples_per_second': 8.585, 'eval_steps_per_second': 0.546}

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Sacrebleu
1,1.874400,1.287148,28.091389
2,1.435400,1.216125,30.835698
3,1.218000,1.220113,30.570394


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

Model fine-tuning complete.


# Test on RisingNews

In [11]:
# Evaluate the model
validation_results = trainer.evaluate()
print("Final validation Metrics", validation_results)

Final validation Metrics {'eval_loss': 1.2161253690719604, 'eval_sacrebleu': 30.835698332906166, 'eval_runtime': 62.604, 'eval_samples_per_second': 9.536, 'eval_steps_per_second': 0.607, 'epoch': 3.0}


In [12]:
# Calculate scores on the test set
final_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Final Test Metrics:", final_test_results.metrics)

Final Test Metrics: {'test_loss': 1.1998350620269775, 'test_sacrebleu': 30.234389725881883, 'test_runtime': 103.8649, 'test_samples_per_second': 9.628, 'test_steps_per_second': 0.607}


In [13]:
print("Final test results:", final_test_results)

Final test results: PredictionOutput(predictions=array([[     2, 250004,   4687, ...,      1,      1,      1],
       [     2, 250004, 100555, ...,      1,      1,      1],
       [     2, 250004, 156637, ...,      1,      1,      1],
       ...,
       [     2, 250004,    360, ...,      1,      1,      1],
       [     2, 250004,    581, ...,      1,      1,      1],
       [     2, 250004,     44, ...,      1,      1,      1]]), label_ids=array([[250004,  43501,  19694, ...,      1,      1,      1],
       [250004,    581,  42998, ...,      1,      1,      1],
       [250004,   1310,    343, ...,      1,      1,      1],
       ...,
       [250004,    360,    903, ...,      1,      1,      1],
       [250004,    581,  43501, ...,      1,      1,      1],
       [250004,   1401,     25, ...,      1,      1,      1]]), metrics={'test_loss': 1.1998350620269775, 'test_sacrebleu': 30.234389725881883, 'test_runtime': 103.8649, 'test_samples_per_second': 9.628, 'test_steps_per_second': 0.60

In [14]:
# Decode the predictions and labels
decoded_predictions = tokenizer.batch_decode(final_test_results.predictions, skip_special_tokens=True)
# decoded_labels = tokenizer.batch_decode(final_test_results.label_ids, skip_special_tokens=True)
labels = np.where(final_test_results.label_ids != -100, final_test_results.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")

Sample Predictions and Labels:

Prediction 1: She was accompanied by the Bangladesh High Commissioner in the United Kingdom at the airport and senior officials of the Bangladesh Mission in London.
Label 1: Bangladesh High Commissioner to the United Kingdom Saida Muna Tasneen and senior officials of Bangladesh Mission in London saw him off at the airport.

Prediction 2: Terrorists have no race, religion or nationality.
Label 2: The terrorists have no race, religion and countries.

Prediction 3: Colonel University Professor Derek Kabera has written an article titled "The Dark Side of Enlightenment."
Label 3: Derek Cabera, professor at Cornell University, wrote an essay called, "The dark sides of categorical thinking."

Prediction 4: He suggested that the public could buy kegs for 45 taka, sugar for 50 taka, mustard for 50 taka and soap oil for 85 taka per liter.
Label 4: He said that people can buy onion at Tk 45 per kg, sugar at Tk 50 kg, lentil at Tk 50 kg and Soybean oil at Tk 85 per 

# Test on SUPara-benchmark

In [15]:
supara_source_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_bn.txt"
supara_target_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_en.txt"

with open(supara_source_test_path, "r", encoding="utf-8") as f:
    supara_source_data = f.readlines()

with open(supara_target_test_path, "r", encoding="utf-8") as f:
    supara_target_data = f.readlines()

print(supara_source_data[0])
print(supara_target_data[0])

সে লাল গোলাপ পছন্দ করে।

She likes red roses.



In [17]:
supara_dataset = Dataset.from_dict({"bn": supara_source_data, "en": supara_target_data})
tokenized_supara_dataset = supara_dataset.map(preprocess_function, batched=True)
print(tokenized_supara_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
# Perform inference with the fine-tuned model
final_test_results_supara = trainer.predict(tokenized_supara_dataset, metric_key_prefix="test")
print("Final test metrics SUPara:", final_test_results_supara.metrics)
print("Final test results SUPara:", final_test_results_supara)

Final test metrics SUPara: {'test_loss': 1.2670142650604248, 'test_sacrebleu': 24.53470013037219, 'test_runtime': 40.834, 'test_samples_per_second': 12.245, 'test_steps_per_second': 0.784}
Final test results SUPara: PredictionOutput(predictions=array([[     2, 250004,   4687, ...,      1,      1,      1],
       [     2, 250004,   1650, ...,      1,      1,      1],
       [     2, 250004,   1401, ...,      1,      1,      1],
       ...,
       [     2, 250004,   2561, ...,      1,      1,      1],
       [     2, 250004,    717, ...,      1,      1,      1],
       [     2, 250004,    262, ...,      1,      1,      1]]), label_ids=array([[250004,   4687,   1884, ...,      1,      1,      1],
       [250004,   1650,  30482, ...,      1,      1,      1],
       [250004,   1401,   6867, ...,      1,      1,      1],
       ...,
       [250004,  50605, 104064, ...,      1,      1,      1],
       [250004,    581,   3486, ...,      1,      1,      1],
       [250004, 106073,   6097, ..., 

In [19]:
# Decode the predicted sequences
decoded_predictions_supara = tokenizer.batch_decode(final_test_results_supara.predictions, skip_special_tokens=True)
labels_supara = np.where(final_test_results_supara.label_ids != -100, final_test_results_supara.label_ids, tokenizer.pad_token_id)
decoded_labels_supara = tokenizer.batch_decode(labels_supara, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions_supara[i]}")
    print(f"Label {i+1}: {decoded_labels_supara[i]}")

Sample Predictions and Labels:

Prediction 1: She likes red pink.
Label 1: She likes red roses.

Prediction 2: It strengthens our bodies.
Label 2: It makes our body firm.

Prediction 3: We live in Bangladesh.
Label 3: We live in Bangladesh.

Prediction 4: Human life is very short.
Label 4: Human life is very short.

Prediction 5: Traffic police have many responsibilities.
Label 5: Traffic police have heavy responsibilities.

Prediction 6: Cricket is my favourite sport.
Label 6: Cricket is my favourite game.

Prediction 7: Public awareness should be created.
Label 7: Mass awareness should be created.

Prediction 8: There may be some problems.
Label 8: There may be some difficulties.

Prediction 9: What's his name?
Label 9: What is her name?

Prediction 10: Our college is very old.
Label 10: Our college is very ancient.
